## Importing libraries

- we are using Tensorflow 2.2 version

In [ ]:
import cv2 #opencv
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


print(tf.__version__)

In [ ]:
foldernames = os.listdir('/kaggle/input/animals10/raw-img')
categories = []
files = []
i = 0
for k, folder in enumerate(foldernames):
    filenames = os.listdir("../input/animals10/raw-img/" + folder);
    for file in filenames:
        files.append("../input/animals10/raw-img/" + folder + "/" + file)
        categories.append(k)
        
df = pd.DataFrame({
    'filename': files,
    'category': categories
})
train_df = pd.DataFrame(columns=['filename', 'category'])
for i in range(10):
    train_df = train_df.append(df[df.category == i].iloc[:500,:])

train_df.head()
train_df = train_df.reset_index(drop=True)
train_df

In [ ]:

! ls ../input/animals10/raw-img

In [ ]:
train_df['filename'].iloc[0]

- peeking into a image

In [ ]:
fig = plt.imshow(plt.imread('../input/animals10/raw-img/pecora/OIP-sLm9oVXQiPp9htcfGmbzpwAAAA.jpeg'))

In [ ]:
img = cv2.imread('../input/animals10/raw-img/pecora/OIP-sLm9oVXQiPp9htcfGmbzpwAAAA.jpeg')
print(img.shape)

In [ ]:
CLASSES = ['cane', 'cavallo', 'elefante', 'farfalla', 'gallina', 'gatto', 'mucca', 'pecora', 'ragno', 'scoiattolo']
train_path = '../input/animals10/raw-img/'


batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150


## Preparing the data

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

## Visualising Images

- Visualize the training images by extracting a batch of images from the training generator—which is 32 images in this example—then plot five of them with matplotlib

In [ ]:
sample_training_images, _ = next(train_data_gen)
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
plotImages(sample_training_images[:5])


## Modelling

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])


In [ ]:
model.compile(optimizer='adam', 
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
total_train = 26179

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
)


In [ ]:
acc = history.history['accuracy']

loss=history.history['loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.title('Training Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.legend(loc='upper right')
plt.title('Training Loss')
plt.show()